In [1]:
## Calculate the potential field kernals for use with the MURaM code. Based on the idl routines of M.C. 2009
import read_muram as rmu
import dp_plot_tools as dplt
import muram_eos as eos
import numpy as np

Importing read_muram.py
Importing dp_plot_tools.py
Importing muram_eos.py


In [2]:
## Vertical is z.

## Simulation Setup

out_dir = '/scratch/przybylski/2D_SCATTER_TESTS/checkerboard/'

## Set Ny = 1 for 2D
## NB dx must = dy for this to work

Nx = 1600
Ny = 1
Nz = 3

dx = 16.0e8/1600
dy = dx
dz = 4.0e8/400

res = 16

heightscale = dz/dx

Nxs = Nx*res
Nys = Ny*res

In [3]:
## Define Fourier grid

kx = np.zeros([Nxs])
ky = np.zeros([Nys])

kx[0:Nxs//2] = np.arange(Nxs//2)/Nxs
kx[Nxs//2:Nxs] = np.arange(Nxs//2)/Nxs-0.5

kx=np.array([kx[:],]*Nys).transpose()  
kx *= 2*np.pi

if Ny > 1:
    ky[0:Nys//2] = np.arange(Nys//2)/Nys
    ky[Nys//2:] = np.arange(Nys//2)/Nys-0.5

ky = np.array([ky[:],]*Nxs)
ky *= 2*np.pi

k2 = kx*kx + ky*ky
kabs = np.sqrt(k2)

In [4]:
## Complex Arrays
HxB = np.zeros([Nxs,Nys],dtype=np.complex)
HyB = np.zeros([Nxs,Nys],dtype=np.complex)
HzB = np.ones([Nxs,Nys],dtype=np.complex)

HxB[np.where(k2 != 0)] = -1j*kx[np.where(k2 != 0)]/kabs[np.where(k2 != 0)]
HxB[0,0] = -1j

if Ny == 1:
    HxB[0,:] = -1j

HyB[np.where(k2 != 0)] = -1j*ky[np.where(k2 != 0)]/kabs[np.where(k2 != 0)]
HyB[0,0] = -1j

if Ny == 1:
    HyB[0,:] = -1j
 

In [5]:
## Delta Function

delta = np.zeros([Nxs,Nys])

for i in range(-4*res,4*res+1):
    i0 = i
    while i0 < 0:
        i0 = i0+Nxs
    for j in range(-4*res,4*res+1):
        j0=j
        while j0 < 0:
            j0 = j0+Nys
        if Ny == 1:
            delta[i0,:] = np.exp(-np.double(np.fmod(i,Nxs))**2/res**2)
        else:
            delta[i0,j0] = np.exp(-(np.double(np.fmod(i,Nxs))**2+np.double(np.fmod(j,Nys))**2)/res**2)

delta/=delta.sum()

In [6]:
## FFT delta function

FFTdelta = np.fft.fft2(delta)

In [7]:
## define function symmetric rebin

def symmetric_rebin(a, oNx, oNy):
    Nx = a.shape[0]
    Ny = a.shape[1]
    dx = Nx/oNx
    dy = Ny/oNy
    
    dxi = np.int(dx)
    dyi = np.int(dy)
    
    out= np.zeros([oNx,oNy])
    b = np.zeros([Nx+2*dxi,Ny+2*dyi])
    
    b[dxi:dxi+Nx,dyi:dyi+Ny] = a
    
    ## Fill y ghost cells
    b[0:dxi,dyi:Ny+dyi] = a[Nx-dxi:Nx,0:Ny]
    b[Nx+dxi:Nx+2*dxi,dyi:Ny+dyi] = a[0:dxi,0:Ny]
    
    ## Fill x ghost cells
    b[:,0:dyi] = b[:,Ny:Ny+dyi]
    b[:,Ny+dyi:Ny+2*dyi] = b[:,dyi:2*dyi]

    cx = np.int(dx/2)
    cy = np.int(dy/2)
    
    for i in range(dxi,Nx+dxi,dxi):
        for j in range(dyi,Ny+dyi,dyi):
            out[i//dxi-1,j//dyi-1] = b[i-cx:i+cx+1,j-cy:j+cy+1].sum()
            
    return out

In [10]:
## Make and Save Kernels:
xbkernel = np.zeros([Nx,Ny])
ybkernel = np.zeros([Nx,Ny])
zbkernel = np.zeros([Nx,Ny])

a = np.zeros(Nx*Ny+3,dtype=np.single)
a[0:3] = [Nx,Ny,heightscale]

for k in range(Nz):
    z = k*heightscale*res
    
    xbkernel[:,:] = symmetric_rebin(np.fft.ifft2(FFTdelta*HxB*np.exp(-kabs*z)).real,Nx,Ny)
    ybkernel[:,:] = symmetric_rebin(np.fft.ifft2(FFTdelta*HyB*np.exp(-kabs*z)).real,Nx,Ny)
    zbkernel[:,:] = symmetric_rebin(np.fft.ifft2(FFTdelta*HzB*np.exp(-kabs*z)).real,Nx,Ny)
    
    flux = zbkernel[:,:].sum()
    
    xbkernel/=flux
    ybkernel/=flux
    zbkernel/=flux
    
    print(k, flux, np.abs(xbkernel[:,:]).sum()/Nx/Ny,np.abs(ybkernel[:,:]).sum()/Nx/Ny,np.abs(zbkernel[:,:]).sum()/Nx/Ny)

    a[3:] = xbkernel[:,:].transpose().ravel() 
    a.tofile(out_dir+'PSF-kernel-x-'+str(k)+'.dat')
    
    a[3:] = xbkernel[:,:].transpose().ravel()
    a.tofile(out_dir+'PSF-kernel-y-'+str(k)+'.dat')
    
    a[3:] = xbkernel[:,:].transpose().ravel() 
    a.tofile(out_dir+'PSF-kernel-z-'+str(k)+'.dat')

0 1.1288993811761299 0.0027765557354221173 0.0 0.000625000000000005
1 1.1289062371731826 0.002390139795771683 0.0 0.000625
2 1.1289062499760465 0.0021732082846600345 0.0 0.0006250000000000001
